In [ ]:
from pandas import DataFrame as df 
import pandas as pd
import numpy as np

In [2]:
Traindata= pd.read_csv('ForecastDataforTraining_201712.csv')

In [3]:
Testdata = pd.read_csv('ForecastDataforTesting_201712.csv')

In [4]:
citydata = pd.read_csv('CityData (1).csv')

In [5]:
windrate = pd.read_csv('In_situMeasurementforTraining_201712.csv')

In [6]:
Testdata6 = Testdata[Testdata.date_id == 6]

In [7]:
location_x = Testdata6.xid.unique().tolist()
location_y = Testdata6.yid.unique().tolist()

In [10]:
tempdict = {}

In [245]:
templist = []
for x in location_x:
    for y in tqdm.tqdm(location_y):
        for hour in range(3,21):
            flag = 0
            temp = Testdata6[(Testdata6.xid==x)&(Testdata6.yid==y)&(Testdata6.hour==hour)]
            temp_wind = temp.wind
            temp_hour = temp.hour
            temp_date = temp.date_id
            if temp_wind.max()>15:
                flag =1
            tempdict = {'xid':x,'yid':y,'dg':flag,'hour':temp_hour,'date':temp_date}
            templist.append(tempdict)
            

  0%|          | 1/421 [00:07<53:28,  7.64s/it]

KeyboardInterrupt: 

In [ ]:
temp = Testdata6[(Testdata6.xid==x)&(Testdata6.yid==y)&(Testdata6.hour==hour)]

In [11]:
a = Testdata6.groupby(['xid','yid','hour'])

In [14]:
citydata

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [15]:
target = [citydata.loc[1].xid,citydata.loc[1].yid]

In [16]:
target

[84, 203]

In [17]:
start = [citydata.loc[0].xid,citydata.loc[0].yid]

In [19]:
Testdata6[(Testdata6.xid==start[0])&(Testdata6.yid==start[1])&(Testdata6.hour==3)]

,xid,yid,date_id,hour,model,wind
596880,142,328,6,3,1,13.9
596881,142,328,6,3,2,12.7
596882,142,328,6,3,3,13.9
596883,142,328,6,3,4,12.6
596884,142,328,6,3,5,13.6
596885,142,328,6,3,6,13.8
596886,142,328,6,3,7,13.5
596887,142,328,6,3,8,13.5
596888,142,328,6,3,9,14.5
596889,142,328,6,3,10,13.8


In [166]:
up = np.array([0,1])
down = np.array([0,-1])
left = np.array([-1,0])
right = np.array([1,0])
total_action = {'up':up,'down':down,'left':left,'right':right,'stay':np.array([0,0])}
crash = {'up':0,'down':0,'left':0,'right':0,'stay':1}

In [167]:
def pathplaning(start,target):
    time = 2
    end = np.array(target)
    location = np.array(start)
    total_time = 0
    while np.abs(np.array(end)-np.array(start)).sum()!=0:
        crash = {'up':0,'down':0,'left':0,'right':0,'stay':0}
        next_location_temps = [location+up,location+down,location+left,location+right,location+np.array([0,0])]
        hope_values = {}
        time_costs = {}
        actions= ['up','down','left','right','stay']
        action = 0
        for next_location_temp in next_location_temps:

            hour = 3+int(time/60)
            #应该先判断是否越界，目前只是雏形，暂时不做判断。
            next_distance_temp = np.abs(end - next_location_temp).sum()
            #print(Testdata6[(Testdata6.xid==next_location_temp[0])&(Testdata6.yid==next_location_temp[1])&(Testdata6.hour==hour)].wind)
            if Testdata6[(Testdata6.xid==next_location_temp[0])&(Testdata6.yid==next_location_temp[1])&(Testdata6.hour==hour)].wind.max()<15:
                next_time_temp = total_time+2
            else:
                next_time_temp = total_time + 1440
                crash[actions[action]] = 1
            if Testdata6[(Testdata6.xid==next_location_temp[0])&(Testdata6.yid==next_location_temp[1])&(Testdata6.hour==(hour+1))].wind.mean()>=15:
                #print(Testdata6[(Testdata6.xid==next_location_temp[0])&(Testdata6.yid==next_location_temp[1])&(Testdata6.hour==(hour+1))].wind.mean())
                alpha = (time-(hour-3)*60)/60
            else:
                alpha =0
            #print(alpha)
            hope_value = next_distance_temp + next_time_temp+1440*alpha
            hope_values[actions[action]] = hope_value
            time_costs[actions[action]] = next_time_temp
            action +=1
        chosen_action = min(hope_values,key=hope_values.get)
        if crash[chosen_action] ==1:
            print(chosen_action,'is crashed')
        
        next_action = location+total_action[chosen_action]
        total_time = time_costs[chosen_action]
        print('location is ',next_action,'time =',time)
        location = next_action
        time +=2
        total_time = time

In [168]:
pathplaning([142,328],[84,203])

location is  [142 327] time = 2
location is  [142 326] time = 4
location is  [142 325] time = 6
location is  [142 324] time = 8
location is  [142 323] time = 10
location is  [142 322] time = 12
location is  [142 321] time = 14
location is  [142 320] time = 16
location is  [142 319] time = 18
location is  [142 318] time = 20
location is  [141 318] time = 22
location is  [141 317] time = 24
location is  [141 316] time = 26
location is  [141 315] time = 28
location is  [141 314] time = 30
location is  [141 313] time = 32
location is  [141 312] time = 34
location is  [141 311] time = 36
location is  [141 310] time = 38
location is  [141 309] time = 40
location is  [140 309] time = 42
location is  [140 309] time = 44
location is  [140 309] time = 46
location is  [140 309] time = 48
location is  [140 309] time = 50
location is  [140 309] time = 52
location is  [140 309] time = 54
location is  [140 309] time = 56
location is  [140 309] time = 58
down is crashed
location is  [140 308] time = 6

KeyboardInterrupt: 

In [117]:
Testdata6[(Testdata6.xid==138) & (Testdata6.yid == 300) & (Testdata6.hour==3)]

,xid,yid,date_id,hour,model,wind
579760,138,300,6,3,1,15.0
579761,138,300,6,3,2,11.8
579762,138,300,6,3,3,13.0
579763,138,300,6,3,4,12.2
579764,138,300,6,3,5,12.2
579765,138,300,6,3,6,13.1
579766,138,300,6,3,7,13.0
579767,138,300,6,3,8,14.7
579768,138,300,6,3,9,13.5
579769,138,300,6,3,10,14.1


In [28]:
x.sum()

2

In [30]:
x[0]

1